In [1]:
## import warnings
import time
import math
import os
import glob
from pyhere import here
from datetime import date

import numpy as np
import pandas as pd
import geopandas
import pickle

import pyarrow
import itertools
import multiprocessing
import p_tqdm

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split, KFold, LeaveOneGroupOut, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import r2_score
from scipy.stats import spearmanr,  pearsonr

In [2]:
hot_encode = True
weighhted_avg = True

In [3]:
f = 'landsat-8-c2-l2_bands-1-2-3-4-5-6-7_ZMB_20k-points_1000-features_yr-2013-2021_mn-4-9_lm-True_cm-True_full.feather'
fn = here('data', 'random_features', 'full_files', f)
features = pd.read_feather(fn)

In [4]:
drop_cols = ['year', 'lon', 'lat', 'crop_perc', 'district']

if weighhted_avg:
    features = features.set_index(drop_cols)
    features.rename(columns={x:y for x,y in zip(features.columns,range(0,len(features.columns)))}, inplace=True)
    features = features.reset_index()
    features.columns = features.columns.astype(str)

if hot_encode:
    drop_cols.remove('district')
    features = pd.get_dummies(features, columns=["district"], drop_first=False)
else:
    pass

features

predictions = features.copy()[drop_cols]

In [6]:
model_fn_suffix = f.replace('_full.feather', '')+ '_wa-True_he-True'
model_fn_suffix 

k_model_fn = f'k-fold-cv_rr-model_{model_fn_suffix}.pkl'
logo_model_fn = f'logo-cv_rr-model_{model_fn_suffix}.pkl'

# with open(here('models', k_model_fn),'wb') as f:
#     pickle.dump(best_kfold_model, f)
        
with open(here('models', k_model_fn), 'rb') as f:
    best_kfold_model = pickle.load(f)

In [7]:
# x_all = features.drop(drop_cols, axis = 1) 
# predictions = best_kfold_model.predict(x_all)

In [8]:
x_all = features.drop(drop_cols, axis = 1) 
predictions['predictions'] = best_kfold_model.predict(x_all)

In [33]:
import seaborn as sns
import matplotlib.pyplot as plt

In [84]:
def scatter(x, y, c, **kwargs):
    del kwargs["color"]
    plt.scatter(x, y, c = c, **kwargs)
    
def yield_plot(**kwargs):
    del kwargs["color"]
    geopandas.GeoDataFrame.plot('yield_mt',**kwargs)

In [64]:
country_shp = geopandas.read_file(here('data', 'geo_boundaries', 'gadm36_ZMB_2.shp'))
country_shp = country_shp.set_index('district')

crop_df = pd.read_csv(here('data', 'crop_yield', 'cfs_maize_districts_zambia_2009_2022.csv'))
crop_df = crop_df[['district', 'year', 'yield_mt']]
crop_df = crop_df.set_index('district')
crop_df = crop_df.join(country_shp).reset_index()
crop_df = geopandas.GeoDataFrame(crop_df)

In [86]:
# g = sns.FacetGrid(
#     crop_df,
#     col='year',
#     col_wrap=3,
#     height=4, 
#     aspect=1
# )
# g.map(yield_plot) 

In [87]:
# g = sns.FacetGrid(
#     predictions,
#     col='year',
#     col_wrap=3,
#     height=4, 
#     aspect=1
# )
# g.map(scatter, 'lon', 'lat', 'predictions', s = .25)    